In [1]:
#Install libraries
!pip install pydicom              
!pip install opencv-python
!pip install pillow
!pip install imantics
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo   
#after installation restart runtime to make libraries available

     |████████████████████████████████| 2.0 MB 9.0 MB/s 
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16031 sha256=ddd7ab16ce27292444dcc0cfbb99220330ce71822dc2148aa8dbbdb94e519273
  Stored in directory: /root/.cache/pip/wheels/da/7c/3e/296fe3ed4eb3bd713e91dee0d0549f12f316d49939a64bdc96
Successfully built imantics
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-vrdz_zb3
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-vrdz_zb3
     |████████████████████████████████| 596 kB 8.8 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65053 sha256=3669330d620ba578560854a9f93c7afc4d734ff3af9e126d5561c3a60e02c0ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-s15qikfe/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
Successfully built fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-

In [1]:
from google.colab import drive        #mount google drive to use it
GDRIVE_ROOT = '/gdrive'
drive.mount(GDRIVE_ROOT, force_remount=True)

Mounted at /gdrive


In [2]:
#Install Remo only after mounting gdrive
!pip install remo

     |████████████████████████████████| 14.3 MB 7.2 MB/s 
     |████████████████████████████████| 148 kB 48.0 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 69 kB 7.3 MB/s 
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 545 kB 51.1 MB/s 
     |████████████████████████████████| 448 kB 50.1 MB/s 
     |████████████████████████████████| 65 kB 3.7 MB/s 
     |████████████████████████████████| 72 kB 865 kB/s 
     |████████████████████████████████| 3.6 MB 38.9 MB/s 
     |████████████████████████████████| 911 kB 40.6 MB/s 
     |████████████████████████████████| 7.5 MB 33.0 MB/s 
     |████████████████████████████████| 133 kB 49.7 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
  Created wheel for django-allauth: filename=django_allauth-0.41.0-py3-none-any.whl size=855166 sha256=00ee4062928217bba3c9629a8d9f152d1e28cb81c790b25e4e1def0628c2a710
  Stored in directory: /root/.ca

In [3]:
#import libraries and dependencies
import pydicom as dicom               
import numpy as np
import glob
import os
import cv2
import PIL
from xml.dom import minidom

import random
from imantics import Polygons, Mask
import torch, torchvision
import detectron2
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader, DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer as Train, DefaultPredictor as Predict
from detectron2.evaluation import inference_on_dataset as Infer, COCOEvaluator as Evaluate
from detectron2.utils.visualizer import Visualizer

In [ ]:
#Convert dcom files into jpg images

dcom_path = glob.glob('/gdrive/MyDrive/INbreast Release 1.0/AllDICOMs/*dcm')
print(dcom_path)    
for image in dcom_path: 
  convert = dicom.dcmread(image)
  pixels = convert.pixel_array.astype(float)
  pixels= 255*pixels/pixels.max()
  image = image.replace('.dcm', '.jpg')
  cv2.imwrite(image, pixels)

In [ ]:
#Extracting info from xmls to form the annotations part of the json file
path2xml= glob.glob('/gdrive/MyDrive/INbreast Release 1.0/AllXML/*.xml')
bboxlist= []
allROI= []
category= []
image_id= []
id=0

for image in path2xml:
  i=3
  s=1
  imgID= image
  imgID= imgID.replace("/gdrive/MyDrive/INbreast Release 1.0/AllXML/","")
  lastdig= len(imgID)
  imgID= (imgID[:lastdig-4]) 
  file = minidom.parse(image)
  numpoints = file.getElementsByTagName('integer')     #will be used to get total number of points (in x,y pixels)
  points = file.getElementsByTagName('string')         #will be used to get a single point (x, y pixels)
  index= numpoints[1].firstChild.data                   #the total number of ROIs in one xml
  #Process all ROI in a single xml
  for x in range(int(index)):                                #store all roi pixel points in array
    image_id.append(imgID)
    category.append(points[s].firstChild.data)
    s= s + int(numpoints[i].firstChild.data) + 1           #the index for string will skip over mm points, and point to first string of pixel points
    allpoints= ''
    
    for y in range(int(numpoints[i].firstChild.data)):       #each point of a single ROI is saved in allpoints
      allpoints= allpoints + points[s+y].firstChild.data + ', '
    
    allpoints= allpoints.replace("(","")
    allpoints= allpoints.replace(")","")
    last= len(allpoints)
    allpoints= (allpoints[:last-2]) 
    allROI.append(allpoints)
    
    #Begin bounding box calculation
    x_leftist=10000
    x_rightest=0
    y_lowest=10000
    y_highest=0
    string= allpoints
    string= string.replace(" ","")
    
    for a in range (int(numpoints[i].firstChild.data)):
      substring= string.split(",")[0]
      if float(substring) < x_leftist:
        x_leftist= float(substring)
      if float(substring) > x_rightest:
        x_rightest= float(substring)
      #string= string.replace(substring,"")
      string= string[(len(substring)+1):]
      substring= string.split(",")[0]
      if float(substring) < y_lowest:
        y_lowest= float(substring)
      if float(substring) > y_highest:
        y_highest= float(substring)
      #string= string.replace(substring,"")
      string= string[(len(substring)+1):]
    
    #Back in main for loop
    width= x_rightest-x_leftist
    height= y_highest-y_lowest
    bbox= str(x_leftist)+', '+ str(y_lowest) +', ' + str(width) +', '+ str(height)
    bboxlist.append(bbox)
    #change indices for tags string and integer
    s= s + int(numpoints[i].firstChild.data) + 1
    i= i + 3


In [ ]:
for ch in range(len(category)):
  if (category[ch]=='Espiculated Region'):
    category[ch]= 'Spiculated Region'
  if (category[ch]=='Calcifications'):
    category[ch]= 'Calcification'
  if (category[ch]=='Spiculated region'):
    category[ch]= 'Spiculated Region'
  if (category[ch]=='Assymetry'):
    category[ch]= 'Asymmetry' 

In [ ]:
#Assign classification labels
category_id= []

for cat in category:
  if cat=='Calcification':
    category_id.append('1')
  elif cat=='Cluster':
    category_id.append('2')
  elif cat=='Mass':
    category_id.append('3') 
  elif cat=='Spiculated Region':
    category_id.append('4')  
  elif cat=='Distortion':
    category_id.append('5')  
  elif cat=='Asymmetry':
    category_id.append('6') 
  elif cat=='Point 1':
    category_id.append('7')
  elif cat=='Point 3':
    category_id.append('8')
  elif cat=='Unnamed':
    category_id.append('9') 
  else:
    print(cat) 


In [ ]:
CocoFile= open('/gdrive/MyDrive/INbreast Release 1.0/coco.json', 'w')

total= len(bboxlist)
CocoFile.write('{\n\t"annotations": [')

for roi_id in range(total):
  x= bboxlist[roi_id]
  x= bboxlist[roi_id]
  annote= '\n\t\t{\n\t\t\t"area": 0,\n\t\t\t"bbox": [\n\t\t\t\t'+bboxlist[roi_id]+'\n\t\t\t],\n\t\t\t"category_id": '+category_id[roi_id]+',\n\t\t\t"id": '+str(roi_id)+',\n\t\t\t"image_id": '+image_id[roi_id]+',\n\t\t\t"iscrowd": 0,\n\t\t\t"segmentation": [\n\t\t\t\t[\n\t\t\t\t'+allROI[roi_id]+'\n\t\t\t\t]\n\t\t\t]\n\t\t},'
  if (roi_id==(total-1)):
    anlast=len(annote)-1
    annote=annote[:anlast]
  CocoFile.write(annote)
CocoFile.write('\n\t],')  


4

In [ ]:
#Categories section
CocoFile.write('\n\t"categories": [')
CocoFile.write('\n\t\t{\n\t\t\t"id": 1,\n\t\t\t"name": "Calcification",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 2,\n\t\t\t"name": "Cluster",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 3,\n\t\t\t"name": "Mass",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 4,\n\t\t\t"name": "Spiculated Region",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 5,\n\t\t\t"name": "Distortion",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 6,\n\t\t\t"name": "Asymmetry",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 7,\n\t\t\t"name": "Point 1",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 8,\n\t\t\t"name": "Point 3",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t},')
CocoFile.write('\n\t\t{\n\t\t\t"id": 9,\n\t\t\t"name": "Unnamed",\n\t\t\t"supercategory": "MammogramFindings"\n\t\t}\n\t],')
   

86

In [ ]:
#we can get image details by going through all images, extract the name, then measure height andd width, make id equal to index of loop
image_path = glob.glob('/gdrive/MyDrive/INbreast Release 1.0/AllDICOMs/*jpg')
CocoFile.write('\n\t"images": [')
z=0
for image in image_path:
  z=z+1
  xy= cv2.imread(image)
  name= image
  name= name.replace("/gdrive/MyDrive/INbreast Release 1.0/AllDICOMs/","")
  name2= name.split("_")[0]
  imagedetails= '\n\t\t{\n\t\t\t"date_captured": "",\n\t\t\t"file_name": "'+name+'",\n\t\t\t"height": '+str(xy.shape[0])+',\n\t\t\t"id": '+name2+',\n\t\t\t"license": 0,\n\t\t\t"url": "",\n\t\t\t"width": '+str(xy.shape[1])+'\n\t\t},'
  if (z==410):
    imlast=len(imagedetails)-1
    imagedetails=imagedetails[:imlast]
  CocoFile.write(imagedetails)

CocoFile.write('\n\t],') 


4

In [ ]:
#ending off information (not important)
CocoFile.write('\n\t"info":\n\t\t{\n\t\t\t"contributor": "",\n\t\t\t"date_created": "",\n\t\t\t"description": "",\n\t\t\t"url": "",\n\t\t\t"version": "",\n\t\t\t"year": ""\n\t\t},')
CocoFile.write('\n\t"licences": [\n\t\t{\n\t\t\t"id": "",\n\t\t\t"name": "",\n\t\t\t"url": ""\n\t\t}\n\t]\n}')

CocoFile.close()


In [4]:
!python -m remo_app init --colab
import remo
remo.set_viewer('jupyter')


    (\(\
    (>':')  Remo: v0.6.1
Python: 3.7.12, Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic

Initiailizing Remo:

This will download and install the following packages as needed: 
   * openssl
   * apt-transport-https
   * ca-certificates
   * unzip
   * libpq-dev
   * python3-dev
   * unzip
   * vips
   * postgres
   * supervisor
   * ngrok

Remo Terms of Service: https://remo.ai/docs/terms/

Do you want to continue with the installation of remo? [Y/n]: Y
Installing Remo to dir: /gdrive/My Drive/RemoApp

[-] Installing OS specific tools
$ sudo apt-get update -qq
$ sudo apt-get install -y -qq openssl
$ sudo apt-get install -y -qq apt-transport-https ca-certificates unzip libpq-dev python3-dev

Completed - components installed
--------------------------------------------------

[-] Installing vips lib
$ sudo apt-get install -y -qq libvips-dev

Completed - components installed
--------------------------------------------------

[-] Installing postgres
$ sudo apt-get install -y -qq 

In [ ]:
#This cell only needs to be run once because the created dataset is stored persistently
imagespath =  '/gdrive/MyDrive/INbreast Release 1.0/AllDICOMs/'             #Path to images
images = glob.glob('/gdrive/MyDrive/INbreast Release 1.0/AllDICOMs/*jpg')   #List of every image file
anno = '/gdrive/MyDrive/INbreast Release 1.0/annotations/'                  #Path to annotations (json file)

random.shuffle(images)                                                      #randomise the order of image files
distribute =  {'training' : images[0:round(len(images)*0.8)], 'testing' : images[round(len(images)*0.8):(len(images)-1)]}   #distribute the images into a set for training and a set for testing
distributionpath = os.path.join(anno, 'distribution.csv')   #Store this distributio in a csv file
remo.generate_image_tags(tags_dictionary  = distribute, output_file_path = distributionpath, append_path = False)           #this tags the images based on the dictionary created
dataset= remo.create_dataset(name = 'INbreast', local_files = [anno, imagespath], annotation_task='Instance Segmentation')  #creates database of images with the instance segmentation masks
print('Statistics Summary:')
dataset.get_annotation_statistics()     #Output statistics for the database
print('Detailed Statistics:')
dataset.view_annotation_stats()

Acquiring data - completed                                                                           
Processing annotation files: 2 of 2 files                                                            Processing data - completed                                                                          
Data upload completed with some errors:
20588020_024ee3569b2605dc_MG_L_CC_ANON.jpg: No valid url
20588072_024ee3569b2605dc_MG_L_ML_ANON.jpg: No valid url
20588680_036aff49b8ac84f0_MG_L_ML_ANON.jpg: No valid url
20587994_024ee3569b2605dc_MG_R_CC_ANON.jpg: No valid url
20588046_024ee3569b2605dc_MG_R_ML_ANON.jpg: No valid url
20588654_036aff49b8ac84f0_MG_R_ML_ANON.jpg: No valid url
50994868_069212ec65a94339_MG_L_CC_ANON.jpg: No valid url
50994895_069212ec65a94339_MG_R_CC_ANON.jpg: No valid url
50994706_069212ec65a94339_MG_R_CC_ANON.jpg: No valid url
50994787_069212ec65a94339_MG_L_CC_ANON.jpg: No valid url
50994733_069212ec65a94339_MG_L_ML_ANON.jpg: No valid url
50994814_069212ec65a94339_MG

In [ ]:
dataset.export_annotations_to_file(anno + 'training.json', annotation_format='coco', filter_by_tags=['training'], export_tags=False, append_path=False)
dataset.export_annotations_to_file(anno + 'testing.json', annotation_format='coco', filter_by_tags=['testing'], export_tags=False, append_path=False)


In [ ]:
register_coco_instances('INbreastTraining', {}, anno + 'training.json', imagespath)
register_coco_instances('INbreastTesting', {}, anno + 'testing.json', imagespath)

In [ ]:
dataset= remo.get_dataset(1)          #If you ran the cell before this, then the dataset has been created and stored. 
                                      #Therefore if you are running this entire program a second time, instead of repeating the cell above, just run this cell

In [ ]:
yaml = './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'
Resnetweights = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl'

model = get_cfg()
model.merge_from_file(yaml)
model.MODEL.WEIGHTS = Resnetweights 
model.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 50
model.MODEL.ROI_HEADS.NUM_CLASSES = 10
model.SOLVER.BASE_LR = 0.002
model.SOLVER.IMS_PER_BATCH = 2   
model.SOLVER.MAX_ITER = 50 
model.DATALOADER.NUM_WORKERS = 2
model.DATASETS.TRAIN = ('INbreastTraining')
model.DATASETS.TEST = ()

os.makedirs(model.OUTPUT_DIR, exist_ok=True)
trainmodel = Train(model)
trainmodel.resume_or_load(resume=False)
trainmodel.train()

[11/22 16:25:30 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:16, 10.7MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due 

[11/22 16:25:54 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/22 16:26:16 d2.utils.events]:  eta: 0:00:29  iter: 19  total_loss: 3.095  loss_cls: 1.058  loss_box_reg: 0.06178  loss_mask: 0.6752  loss_rpn_cls: 0.39  loss_rpn_loc: 0.5779  time: 1.0288  data_time: 0.8104  lr: 0.00076124  max_mem: 2283M
[11/22 16:26:36 d2.utils.events]:  eta: 0:00:10  iter: 39  total_loss: 1.674  loss_cls: 0.3552  loss_box_reg: 0.3089  loss_mask: 0.5363  loss_rpn_cls: 0.09338  loss_rpn_loc: 0.3653  time: 1.0278  data_time: 0.7669  lr: 0.0015604  max_mem: 2283M
[11/22 16:26:48 d2.utils.events]:  eta: 0:00:00  iter: 49  total_loss: 1.393  loss_cls: 0.2067  loss_box_reg: 0.2327  loss_mask: 0.5398  loss_rpn_cls: 0.08258  loss_rpn_loc: 0.3297  time: 1.0249  data_time: 0.7623  lr: 0.00196  max_mem: 2283M
[11/22 16:26:49 d2.engine.hooks]: Overall training speed: 48 iterations in 0:00:49 (1.0249 s / it)
[11/22 16:26:49 d2.engine.hooks]: Total training time: 0:00:51 (0:00:02 on hooks)


In [ ]:
eval = Evaluate('INbreastTesting', model, False, output_dir='./output/')
loader = build_detection_test_loader(model, 'INbreastTesting')
results = Infer(trainmodel.model, loader, eval)
print('Evaluation:')
print(results)

WARNING [11/22 16:27:17 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [11/22 16:27:18 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/22 16:27:18 d2.data.datasets.coco]: Loaded 61 images in COCO format from /gdrive/MyDrive/INbreast Release 1.0/annotations/testing.json
WARNING [11/22 16:27:18 d2.data.datasets.coco]: Filtered out 912 instances without valid segmentation. There might be issues in your dataset generation process.  Please check https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html carefully
[11/22 16:27:18 d2.data.build]: Distribution of instances among all 6 categories:
|   category    | #instances   |  category  | #instances   |  category  | #instances   |
|:-------------:|:-------------|:----------:|:-------------|:----------:|:-------------|
| calcification | 657          |  cluste

Exception: ignored

In [ ]:
model.MODEL.WEIGHTS = os.path.join(model.OUTPUT_DIR, 'model_final.pth')
model.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
model.DATASETS.TEST = ('INbreastTesting',)
predictmodel = Predict(model)

In [ ]:
testdataset = DatasetCatalog.get('INbreastTesting')
metadata = MetadataCatalog.get('INbreastTraining')
classdict = dict(enumerate(metadata.thing_classes))
annotations = []
for item in testdataset:
  name = item['file_name']
  image= np.array(PIL.Image.open(name))
  prediction = predictmodel(image)
  classes = prediction['instances'].get('pred_classes').cpu().numpy()
  masks = outputs['instances'].get('pred_masks').cpu().permute(1, 2, 0).numpy()
  
  dim = 1
  if masks.shape[2] != 0:
    dim= masks.shape[2]
  for x in range(dim):
    polygons = Mask(masks[:, :, x]).polygons()
    a = remo.Annotation()
    a.img_filename = name
    a.classes = classdict[classes[x]]
    a.segment = polygons.segmentation[0]
    annotations.append(a)

WARNING [11/22 16:34:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[11/22 16:34:39 d2.data.datasets.coco]: Loaded 61 images in COCO format from /gdrive/MyDrive/INbreast Release 1.0/annotations/testing.json
WARNING [11/22 16:34:39 d2.data.datasets.coco]: Filtered out 912 instances without valid segmentation. There might be issues in your dataset generation process.  Please check https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html carefully


ValueError: ignored

In [ ]:
PredictedMasks = dataset.create_annotation_set(annotation_task = 'Instance Segmentation', name = 'Mask Prediction')
dataset.add_annotations(annotations, annotation_set_id=PredictedMasks.id)
dataset.view()

In [ ]:
remo.open_ui()